## Here In This Notebook there are the implementation of the Naive Bayesian Classifier and the Logistic Regressor for the Multi Class Classification on the K-MNIST Dataset.

Assignment-1
P4
a)

In [ ]:
import numpy
from numpy import *
import matplotlib.pyplot as plot

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

## Making the Data to be Ready

In [ ]:
data_raw=genfromtxt("/content/drive/MyDrive/Data/PRNN/Assignment_1/Kannada-MNIST/train.csv",delimiter=",")

In [ ]:
X_raw,Y_raw=data_raw[1:,1:],data_raw[1:,0]
X_raw=X_raw/255
Y_raw=Y_raw.astype(int)
N,d=shape(X_raw)
print(N,d)

In [ ]:
Ns=6000  #samples_of_choice

random_index=random.choice(arange(N),Ns)

In [ ]:
Y_raw

In [ ]:
X_raw[random_index].shape

In [ ]:
noise = random.rand(Ns,d)
X_raw=X_raw[random_index]+0.1*noise
Y_raw=Y_raw[random_index]

In [ ]:
X1=(X_raw[0,:])#+0.1*noise[0,:]

#X1=X_raw[0,:].astype(int)
X1/255
X1.shape

#plot.imshow(resize((X1/255),(28,28)),cmap="gray")

In [ ]:
train_test_split_ratio=0.9
train_split=train_test_split_ratio*float(Ns)
test_split=round((1-train_test_split_ratio)*float(Ns))

In [ ]:
# train_index=random.choice(arange(N),int(train_split))
# train_index.shape
# test_index=
test_split

In [ ]:
train_X,train_Y=X_raw[0:int(train_split)],Y_raw[0:int(train_split)]
test_X,test_Y=X_raw[Ns-int(test_split):Ns],Y_raw[Ns-int(test_split):Ns]

In [ ]:
test_Y

In [ ]:
K=10
labels=array([range(0,K)])
C=[]
X_tr=[]
for j in range(K):

  C_j=where(train_Y==j)         #set of indexes where a particular class Y==j
  c_j=len(C_j[0])                  #Cardinality of that set
  C.append(c_j)
  X_tr.append(train_X[C_j,:])
C_j
numpy.sum(C)

In [ ]:
X_train=X_tr

In [ ]:
X_train[0].shape
X_train[0][0]
Xk=reshape(X_train[0][0][1],(1,d))
Xk.shape
# for j in range(10):
#   print(j)
#   j=j+1
# j

In [ ]:
X_train[0][0][0].shape

## Calculating the Sample Means and Sample Covariances for each of the classes

In [ ]:
def SecondOrderStatistics(training_data):

  Sample_mean=[]
  Sample_covariances=[]
  sum1,sum2=0,0

  for j in range(K):
    for jj in range(C[j]):
      X_i=reshape(training_data[j][0][jj],(1,d))
      sum1=sum1+(X_i)
    Sample_mean.append(sum1/C[j])
    sum1=0
    for ii in range(C[j]):
      X_ii=reshape(training_data[j][0][ii],(1,d))
      sum2=sum2+(X_ii-Sample_mean[j]).T * (X_ii-Sample_mean[j])
    Sample_covariances.append(sum2/C[j])
    sum2=0
  return Sample_mean,Sample_covariances

#Basically building the Multivariate Gaussians for each classes

In [ ]:
Sample_mean,Sample_covariances = SecondOrderStatistics(X_train)
Sample_mean,Sample_covariances=array(Sample_mean),array(Sample_covariances)
print(Sample_mean.shape,Sample_covariances.shape)

In [ ]:
plot.imshow(reshape(Sample_mean[random.choice(10,1)],(28,28)),cmap="gray")

In [ ]:
X_train[0][0][0][0].shape
X_train[0].shape

Since we are building the Naive Bayes Classifier, we take all our pixels to be independent and that's why our covariance matrix will be diagonal, and , here is the code for the same

In [ ]:
def NaiveCovarianceMatrix(training_data,sample_mean):
  naive_covariance_mat=[]
  #temp=zeros((d,d))
  for j in range(K):
    X_mi=training_data[j]
    sum=0
    temp=zeros((d,d))
    for m in range(d):
      sum=dot((X_mi[0,:,m].reshape(1,C[j])-sample_mean[j,0,m]),(X_mi[0,:,m].reshape(C[j],1)-sample_mean[j,0,m]))
      temp[m,m]=sum/C[j]
    naive_covariance_mat.append(temp)
  return naive_covariance_mat

In [ ]:
naive_covariance_mat = NaiveCovarianceMatrix(X_train,Sample_mean)
naive_covariance_mat=array(naive_covariance_mat)
naive_covariance_mat.shape

In [ ]:
naive_covariance_mat.shape

In [ ]:
diag=[]

for j in range(K):
  for m in range(d):
    diag.append(naive_covariance_mat[j,m,m])
diag=array(diag)
diag=diag.reshape(10,1,784)
diag[0]
diag[0].shape



In [ ]:
test_X[0,:].shape

Since, the dimension is large det(naive_covariance_mat) = 0,
and hence, we go by taking the log of the likelyhood so that the log(det(naive_covariance_mat)) = numpy.sum(log(diag))

In [ ]:
def loglikelyhoods(testing_data,sample_mean,naive_covariance_matrix):
  log_likelyhood_mat=zeros((K,int(testing_data.shape[0])))
  constant=-(testing_data.shape[1]/2)*log(2*pi)
  diag=[]

  for j in range(K):
    for m in range(testing_data.shape[1]):
      diag.append(naive_covariance_matrix[j,m,m])
  diag=array(diag)
  diag=diag.reshape(K,1,testing_data.shape[1])
  for j in range(K):
    for m in range(int(test_split)):
      x_m=reshape(testing_data[m,:],(1,testing_data.shape[1]))
      log_likelyhood_mat[j,m]=(constant+((-1/2)*numpy.sum(log(diag[j])))-
       ((testing_data[m,:].reshape(1,testing_data.shape[1])-Sample_mean[j]) @ linalg.inv(naive_covariance_mat[j]) @ (testing_data[m,:].reshape(testing_data.shape[1],1)-Sample_mean[j].reshape(testing_data.shape[1],1))))
  return log_likelyhood_mat

In [ ]:
c=0
for i in range(d):
  for j in range(d):
    if i==j and naive_covariance_mat[3,i,j]!=0:
     c=c+1
print(c)

Testing our Data!!!

In [ ]:
log_likelihood_mat = loglikelyhoods(test_X,Sample_mean,naive_covariance_mat)
y_pred=[]
for j in range(int(test_split)):
  m_index=where(log_likelihood_mat[:,j]==max(log_likelihood_mat[:,j]))
  y_pred.append(array(m_index))
y_pred=array(y_pred)


In [ ]:
print(y_pred.shape,shape(test_Y))


In [ ]:
c=0
for i in range(int(test_split)):
  if y_pred[i]-test_Y[i]!=0:
    c=c+1
accuracy=(1-(c/test_split))*100
print("accuracy:",accuracy)

In [ ]:
shape(labels)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_mattrix=confusion_matrix(y_pred[:,:,0],test_Y)
confusion_mattrix

#Sampling from the Estimated Distributions:
To demonstrate its Generative Nature.

Since, we have assumed the distributions and have got the Parameters by ML estimate , now our this model can gererate the samples.!!!

In [ ]:
#Sample from the Estimated Gaussian
x = random.multivariate_normal(mean = Sample_mean[1][0], cov = Sample_covariances[1])

In [ ]:
plot.imshow(x.reshape(28,28),cmap = "gray")
plot.show()

b) Logistic Regression With Gradient Descent

In [ ]:
X_raw,Y_raw=data_raw[1:,1:],data_raw[1:,0]
X_raw,Y_raw=X_raw/255, Y_raw.astype(int)
N,d=shape(X_raw)

In [ ]:
labels=unique(Y_raw)
labels.shape

In [ ]:
Ns=10000
random_samples=random.choice(arange(N),Ns)

In [ ]:
X_raw1,Y_raw1=X_raw[random_samples],Y_raw[random_samples]

In [ ]:
Y_raw1.shape

In [ ]:
train_test_split_ratio=0.9

train_split=int(train_test_split_ratio*Ns)
test_split=int(Ns-train_split)

In [ ]:
train_split

In [ ]:
print(X_raw1.shape)
bias_vector=ones((Ns,1))
X_raw2=concatenate((X_raw1,bias_vector),axis=1)


X_raw2.shape
X_train,X_test=X_raw2[0:train_split],X_raw2[Ns-test_split:Ns]

X_train.shape,X_test.shape

In [ ]:
plot.imshow(X_train[0,0:d].reshape(28,28),cmap="gray")#.shape

In [ ]:
Y_raw_onehot=zeros((Ns,len(labels)),int)

for j in (labels):
  for i in range(Ns):
    if Y_raw1[i]==j:
      Y_raw_onehot[i,j]=1



In [ ]:
Y_raw_onehot[1]

In [ ]:
Y_train,Y_test=Y_raw_onehot[0:train_split],Y_raw_onehot[train_split:Ns]

In [ ]:
W=ones((len(labels),d+1))


In [ ]:
Z=zeros((len(labels),train_split))

for j in (labels):
  for i in range(train_split):
    Z[j,i]=dot(W[j,:].reshape(1,d+1),X_train[i,:].reshape(d+1,1))


In [ ]:
dot(W[3,:].reshape(1,d+1),X_train[0,:].reshape(d+1,1))

In [ ]:
numpy.sum(exp(Z[:,0]))

In [ ]:
H=zeros((len(labels),train_split))

for i in range(train_split):
  for j in (labels):
    H[j,i]=exp(Z[j,i])/numpy.sum(exp(Z[:,i]))


In [ ]:
H

In [ ]:
array_0=[0,1]#,9,8]

H[:,array_0]

In [ ]:
numpy.sum(H[:,0])

In [ ]:
def empirical_risk(Y,LOGH):
  sum=0

  for i in range(train_split):

    sum=sum-dot(reshape(Y_train[i,:],(1,len(labels))),reshape(log(H[:,i]),(len(labels),1)))

  return sum

In [ ]:
risk=empirical_risk(Y_train,H)

In [ ]:
risk

In [ ]:
def gradient(X_train,Y_train,W_vector,H,m):
  train_split=len(X_train[:,0])
  sum=0

  for i in range(train_split):
    h_mat=zeros((len(labels),d+1))
    # sum=0

    for j in (labels):
      if j!=m :
        h_mat[j,:]=X_train[i,:]*H[m,i]
      elif j==m:
        h_mat[j,:]=X_train[i,:]*(H[m,i]-1)
    sum = sum +dot(Y_train[i,:].reshape(1,len(labels)),h_mat)

  return sum




def stochastic_gradient(H,number_of_samples):

  iterations=1000
  Batch=number_of_samples
  # X_Batch=X_train[Batch]
  # H_Batch=H
  Ones=ones((len(Batch),1))

  for i in range(1000):
    sum1,sum2=0,0
    for b in (Batch):
      sum1=sum1+X_train[b]*(1-H[b])
    sum2=sum2+sum1

  return sum2/1000





In [ ]:
W=ones((len(labels),d+1))

Z=zeros((len(labels),train_split))

for j in (labels):
  for i in range(train_split):
    Z[j,i]=dot(W[j,:].reshape(1,d+1),X_train[i,:].reshape(d+1,1))

H=zeros((len(labels),train_split))

for i in range(train_split):
  for j in (labels):
    H[j,i]=exp(Z[j,i])/numpy.sum(exp(Z[:,i]))



alpha=0.0001  #Learning Rate:

for ii in range(10000):

  # if ii==0:
  #risk_old=risk

  risk_old=empirical_risk(Y_train,H)
  # random_samps=random.choice(arange(train_split),100)
  for m in (labels):
    W[m,:]=W[m,:].reshape(1,d+1)-alpha*gradient(X_train,Y_train,W[j,:].reshape(1,d+1),H,m)
  # if ii==0:
  #   break



  # if ii==0:
  #   print(Z)
  for jj in ((labels)):
    for iii in range(train_split):
      Z[jj,iii]=dot(W[jj,:].reshape(1,d+1),X_train[iii,:].reshape(d+1,1))

  # if ii==0:
  #   print(H)
  for jjj in ((labels)):
    for iiii in range(train_split):
      H[jjj,iiii]=exp(Z[jjj,iiii])/numpy.sum(exp(Z[:,iiii]))
  # if ii==0:
  #   print(H)

  # # if ii==0:
  # #  risk_old=risk
  # if ii==0:
  #   print(risk_old)
  risk_new=empirical_risk(Y_train,H)
  print(risk_old,risk_new)
  # if ii==0:
  #   print(risk_new)
  if abs(risk_new-risk_old)<=10**-1:# or ii==5:
    break
print(ii)


In [ ]:
 W[0]
 risk_new
 Z
 H

In [ ]:
y_pred=[]

for i in range(train_split):
  max_index=where(H[:,i]==max(H[:,i]))
  y_pred.append(int(array((max_index))))

In [ ]:
Z_test=zeros((len(labels),test_split))

for i in range(test_split):
  for j in (labels):
    Z_test[j,i]=dot(W[j,:].reshape(1,d+1),X_test[i,:].reshape(d+1,1))


In [ ]:
H_test=zeros((len(labels),test_split))

for i in range(test_split):
  for j in (labels):
    H_test[j,i]=exp(Z_test[j,i])/sum(exp(Z_test[:,i]))


In [ ]:
y_pred=[]

for i in range(test_split):
  max_index=where(H_test[:,i]==max(H_test[:,i]))
  y_pred.append(max_index)


y_pred=array(y_pred)
y_pred[2]
Y_test=Y_raw1[Ns-test_split:Ns]

In [ ]:
y_pred[:,0,0]-Y_test
Y_test.shape
y_pred[:,0,0].shape

In [ ]:
c=0

for i in range(test_split):
  if y_pred[i,0,0]-Y_test[i]!=0:
    c=c+1

accuracy=(1-c/Ns)*100
accuracy